In [ ]:

from poitousprint import get_online_csv, Toflit
from vega import VegaLite
import pandas as pds

toflit_client = Toflit()

#origins = get_online_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSKCt7D4NgIN-5UIbwesdTyhhze2dVt0oovodSIu_sdTxGueFJ3EgkpYxh9_gIqXBQ1YjQFYppLxAOo/pub?output=csv')
#regions_province = get_online_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSDyf3-ZL7AjZTal84kEwXThfchUW8K9XzEBSzex0H6RzuQkB5u19x9xnHhEBmcSLZBGCk5EY5gJji-/pub?gid=1984675165&single=true&output=csv')
#origin_by_province = {o["origin"]:o["province"] for o in origins}
#origin_by_province['Inconnu'] = 'Inconnu'
#origin_by_province['Colonies Françaises'] = 'Colonies Françaises'
#origin_by_province.update({o["origin_norm_ortho"]:o["province"] for o in origins})
#customs_region_by_province = {o["customs_region_simpl"]:o["hinterland"] for o in regions_province if o["hinterland"] !='' and o["customs_region_grouping"] != ""}
#origins_LaRochelle = set([o["origin_norm_ortho"] for o in origins if o["province"] in [
        #"Aunis", "Poitou", "Angoumois", "Saintonge"]])
#origins_LaRochelle_sans_augoumois = set([o["origin_norm_ortho"] for o in origins if o["province"] in [
 #       "Aunis", "Poitou", "Saintonge"]])

origin_LaRochelle = lambda f: f['origin_province'] in ["Aunis", "Poitou", "Angoumois", "Saintonge"]

In [ ]:

# tous les flux de 89 direction pas la Rochelle et exports
all_flows = toflit_client.get_flows(best_guess_region_prodxpart='1', year='1789', export_import='Exports')

# compléter les origines
origin_provinces = set()
for f in all_flows :
    if f['origin'] == "" and f['product_reexportations'] == "Réexportation":
        f['origin_province'] = "Colonies Françaises"
    if f["origin"] == "":
         f['origin_province'] = "Inconnu"
    origin_provinces.add(f['origin_province'])


In [ ]:
print(sorted(list(origin_provinces)))

In [ ]:
values_per_direction = {}

for f in all_flows:
    direction = f['customs_region']
    value = f['value'] if f['value'] != '' else 0
    # value = int(value.split('.')[0])
    itype = f['export_import']
    year = f['year']
    if itype == 'Exports':
        if direction not in values_per_direction:
            values_per_direction[direction] = 0
        values_per_direction[direction] += float(value)
# parmi tous les exports des directions quelle est la part qui vient de La Rochelle
def process_flow_against_total_per_region(flow):
    f = flow.copy()
    if f['origin'] != '' and origin_LaRochelle(f):
        f['origin_poitou'] = 'De la région de La Rochelle'
    elif f['origin'] == '':
        f['origin_poitou'] = "Pas d'origine"
    else:
        f['origin_poitou'] = 'Hors de la région de La Rochelle'
    f['value'] = float(f['value']) if f['value'] != '' else 0
    f['value_part'] = f['value'] / values_per_direction[f['customs_region']]
    return f
values_per_direction_la_rochelle = 0

for f in all_flows:
    direction = f['customs_region']
    value = f['value'] if f['value'] != '' else 0
    # value = int(value.split('.')[0])
    itype = f['export_import']
    year = f['year']
    if itype == 'Exports' and origin_LaRochelle(f):
        values_per_direction_la_rochelle += float(value)
        
def process_flow_rel_region(flow):
    f = flow.copy()
    if f['origin'] != '' and origin_LaRochelle(f):
        f['origin_poitou'] = 'De la région de La Rochelle'
    elif f['origin'] == '':
        f['origin_poitou'] = "Pas d'origine"
    else:
        f['origin_poitou'] = 'Hors de la région de La Rochelle'
    f['value'] = float(f['value']) if f['value'] != '' else 0
    f['value_part'] = f['value'] / values_per_direction_la_rochelle
    return f


In [ ]:


VegaLite({
    "title": "Part des échanges de produits de La Rochelle par rapport au total des valeurs des exports par région",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "nominal", 
            "field": "origin_poitou",
            "title": "origin_poitou"
        },
        "y": {
            "field": "customs_region",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": "-x"
        },
        "color": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "Part des échanges de produits de La Rochelle par rapport au total des valeurs des exports par région"
        }
    }
}, pds.DataFrame([process_flow_against_total_per_region(flow) for flow in all_flows]))

In [ ]:


VegaLite({
    "title": "Part des échanges de produits de La Rochelle par rapport au total des produits de La Rochelle",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "nominal", 
            "field": "origin_poitou",
            "title": "origin_poitou"
        },
        "y": {
            "field": "customs_region",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": "-x"
        },
        "color": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "Part des échanges de produits de La Rochelle par rapport au total des produits de La Rochelle"
        }
    }
}, pds.DataFrame([process_flow_rel_region(flow) for flow in all_flows]))

In [ ]:


VegaLite({
    "title": "Part des valeurs des produits originaires de la Rochelle exportés par les autres directions des fermes",
    "mark": "bar",
    "encoding": {
        "y": {
            "type": "nominal", 
            "field": "customs_region",
            "title": "direction des fermes",
            "sort": "-x"
        },
        "x": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "Portion des valeurs par rapport au total des exports impliquant des produits de la région"
        }
    }
}, pds.DataFrame([process_flow_rel_region(flow) for flow in all_flows if origin_LaRochelle(flow) and flow['customs_region'] != 'La Rochelle']))

In [ ]:


VegaLite({
    "title": "Part des exports rapportée au total des produits par région",
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal", 
            "field": "product_revolutionempire",
            "title": "type",
            "sort": "-size"
        },
        "y": {
            "field": "customs_region",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": "-size"
        },
        "color": {
            "field": "origin_poitou",
            "type": "nominal"
        },
        "size": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "Part des exports rapportée au total des produits de la région"
        }
    }
}, pds.DataFrame([process_flow_against_total_per_region(flow) for flow in all_flows if process_flow_against_total_per_region(flow)['origin_poitou'] == "De la région de La Rochelle" and flow['customs_region'] != 'La Rochelle']))

In [ ]:



VegaLite({
    "title": "Part des exports de produits originaires de La Rochelle dans les autres directions (relatif au total des produits de la région)",
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal", 
            "field": "product_revolutionempire",
            "title": "type de produit",
            "sort": "-size"
        },
        "y": {
            "field": "customs_region",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": "-size"
        },
        "color": {
            "field": "origin_poitou",
            "type": "nominal"
        },
        "size": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "Part des exports par rapport à la valeur totale des produits de la région exportés par la France"
        }
    }
}, pds.DataFrame([process_flow_rel_region(flow) for flow in all_flows if process_flow_rel_region(flow)['origin_poitou'] == "De la région de La Rochelle" and flow['customs_region'] != 'La Rochelle']))

In [ ]:
exports_small_multiples = [process_flow_rel_region(flow) for flow in all_flows if process_flow_rel_region(flow)['origin_poitou'] == "De la région de La Rochelle" and flow['customs_region'] != 'La Rochelle']
exports_small_multiples = sorted(exports_small_multiples, key=lambda f : f["value_part"])
VegaLite({
    "title": "Part des exports de produits originaires de La Rochelle dans les autres directions (petits multiples)",
    "mark": "bar",
    "encoding": {
        "row": {
            "field": "customs_region",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": {
                "op": "sum",
                "field": "value_part",
                "order": "descending"
            }
        },
        "x": {
            "type": "nominal", 
            "field": "product_revolutionempire",
            "title": "type de produit",
            "sort": "-y"
        },
        "y": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "part des échanges"
        },
        "color": {
            "field": "product_revolutionempire",
            "type": "nominal",
            "sort": {
                "op": "sum",
                "field": "value_part",
                "order": "descending"
            }
        }
    }
}, pds.DataFrame(exports_small_multiples))

# Top des produits d'origine Direction des fermes La Rochelle

In [ ]:
from itertools import groupby
from collections import defaultdict

DFLR_product_direction = []
total_value_by_product = defaultdict(lambda : 0)

DFLR_origin_flows = (f for f in all_flows if origin_LaRochelle(f))
by_product = lambda f: f['product_revolutionempire']
for product, flows_by_product in groupby(sorted(DFLR_origin_flows, key = by_product), key = by_product):
    by_direction = lambda f: f['customs_region']
    for region, flows_by_product_by_region in groupby(sorted(flows_by_product, key=by_direction), key=by_direction):
        flows_by_product_by_region = list(flows_by_product_by_region)
        total_value = sum([float(f['value']) for f in flows_by_product_by_region if f['value'] != ""])
        partners = list(set((f['partner_grouping'] for f in flows_by_product_by_region)))
        total_value_by_product[product]+=total_value
        DFLR_product_direction.append({
            "product":product, 
            "region":region,
            "value":total_value, 
            "partners": partners })
DFLR_product_direction.sort(key = lambda f:f['value'],reverse=True)
print("\n".join((f"{f['region']}->{f['product']}:{f['value']}" for f in DFLR_product_direction[:20])))

In [ ]:
# extract top by value
nb_top_value = 30
top_products = [k for k,v in sorted(total_value_by_product.items(), key=lambda kv:kv[1], reverse=True)[:nb_top_value]]
DFLR_products_region_share = [dict(list(pd.items()) + [("share", 100*pd['value']/total_value_by_product[pd['product']])]) for pd in DFLR_product_direction if pd['product'] in top_products]
VegaLite({
  "title": f"Top {nb_top_value} en valeur des produits d'origine DFLR exportés par direction des fermes en 1789",
  "mark": {"type":"rect", "tooltip": {"content": "data"}},
  "encoding": {
    "x": {
      "field": "region",
      "title": "Directions des fermes",
      "type": "ordinal",
      "sort": {"op":"sum", "field":"value", "order": "descending"}
    },
    "y": {
      "title": f"Top {nb_top_value} product",
      "field":"product",
      "type": "ordinal",
      "sort": {"op":"sum", "field":"value", "order": "descending"}
    },
    "color": {
      "field": "share",
      "title": "share on total product",
      "type": "quantitative",
      #"scale": {"type": "sqrt"}
    }
  },
  "config": {
    "view": {
      "stroke": "transparent"
    }
  }
}, pds.DataFrame(DFLR_products_region_share))

In [ ]:
from itertools import groupby
from collections import defaultdict

DFLR_product_region_partner = []

DFLR_origin_flows = (f for f in all_flows if origin_LaRochelle(f))
by_region = lambda f: f['customs_region']
by_product = lambda f: f['product_revolutionempire']
for product, flows_by_product in groupby(sorted(DFLR_origin_flows, key = by_product), key = by_product):
    for region, flows_by_region in groupby(sorted(flows_by_product, key = by_region), key = by_region):
        by_partner = lambda f: f['partner_grouping']
        for partner, flows_by_region_by_partner in groupby(sorted(flows_by_region, key=by_partner), key=by_partner):
            flows_by_region_by_partner = list(flows_by_region_by_partner)
            total_value = sum([float(f['value']) for f in flows_by_region_by_partner if f['value'] != ""])
            DFLR_product_region_partner.append({
                "product": product,
                "partner":partner, 
                "region":region,
                "value":total_value})
DFLR_product_region_partner.sort(key = lambda f:f['value'],reverse=True)
print("\n".join((f"{f['region']}->{f['partner']}:{f['value']}" for f in DFLR_product_region_partner[:20])))


In [ ]:
nb_top_value = 10
top_products = [k for k,v in sorted(total_value_by_product.items(), key=lambda kv:kv[1], reverse=True)[:nb_top_value]]
DFLR_products_partner_region_share = [dict(list(pd.items()) + [("share", 100*pd['value']/total_value_by_product[pd['product']])]) for pd in DFLR_product_region_partner if pd['product'] in top_products]

VegaLite({
  "title": f"Top {nb_top_value} en valeur des produits d'origine DFLR exportés par direction des fermes en 1789",
  "mark": {"type":"rect", "tooltip": {"content": "data"}},
  "encoding": {
    "row":{"field":"product", "sort": {"op":"sum", "field":"value", "order": "descending"}},
    "x": {
      "field": "region",
      "label": "Directions des fermes",
      "type": "ordinal",
         "axis": {
          "orient":"top" },
      "sort": {"op":"sum", "field":"value", "order": "descending"}
    },
    "y": {
      "label": f"Partenaire commercial",
      "field":"partner",
      "type": "ordinal",
      "sort": {"op":"sum", "field":"value", "order": "descending"}
    },
    "color": {
      "field": "share",
      "type": "quantitative",
      #"scale": {"type": "sqrt"}
    }
  },
"resolve": {"axis":{"x":"independant"}},
  "config": {
    "view": {
      "stroke": "transparent"
    }
  }
}, pds.DataFrame(DFLR_products_partner_region_share))

# Matrice province d'origine direction des fermes

In [ ]:

from itertools import groupby
from collections import defaultdict

DFLR_product_region_partner = []
total_value_by_province = defaultdict(lambda:0)
total_value_by_region = defaultdict(lambda:0)

by_region = lambda f: f['customs_region']
for region, flows_by_region in groupby(sorted((f for f in all_flows), key = by_region), key = by_region):
    by_province_origin = lambda f: f['origin_province']
    for province, flows_by_province_origin in groupby(sorted(flows_by_region, key=by_province_origin), key=by_province_origin):
            flows_by_province_origin = list(flows_by_province_origin)
            total_value = sum([float(f['value']) for f in flows_by_province_origin if f['value'] != ""])
            total_value_by_province[province] += total_value
            total_value_by_region[region] += total_value
            DFLR_product_region_partner.append({
                "province_origine": province,
                "region":region,
                "province_region": flows_by_province_origin[0]["customs_region_province"],
                "value":total_value})
            
for province, value in total_value_by_province.items():
    DFLR_product_region_partner.append({
    "province_origine": province,
    "region": "total",
    "province_region": "total",
    "value":value})
total_value_by_province["total"] = sum(v for k,v in total_value_by_province.items())
for region, value in total_value_by_region.items():
    DFLR_product_region_partner.append({
    "province_origine": "total",
    "region": region,
    "province_region": "?",
    "value":value})
    
total_value_by_region["total"] = sum(v for k,v in total_value_by_region.items())
DFLR_product_region_partner.sort(key = lambda f:f['value'],reverse=True)
print("\n".join((f"{f['region']}->{f['province_origine']}:{f['value']}" for f in DFLR_product_region_partner[:20])))

In [ ]:
# extract top by value
nb_top_value = 1000
top_provinces = [k for k,v in sorted(total_value_by_province.items(), key=lambda kv:kv[1], reverse=True)[:nb_top_value]]
top_provinces += ['total']
DFLR_product_region_partner_share=[]
for pd in DFLR_product_region_partner:
  if pd['province_origine'] in top_provinces:
    sort_meta = {
      "share": 100*pd['value']/total_value_by_province[pd['province_origine']],
      "region_sort": top_provinces.index(pd['province_region']) if pd['province_region'] in top_provinces else nb_top_value,
      "province_sort": top_provinces.index(pd['province_origine'])
    }
    DFLR_product_region_partner_share.append(
      dict(list(pd.items())+list(sort_meta.items())))
#DFLR_product_region_partner_share.append({"share":0, "region_sort":0, "province_sort":0, "region": "Lyon", "province_origine": "Lyonnais"})
VegaLite({
  "title": f"Part des exports (en valeur) par province d'origine et par direction des fermes en 1789",
  "mark": {"type":"rect", "tooltip": {"content": "data"}},
  "encoding": {
    "x": {
      "field": "region",
      "title": "Directions des fermes",
      "type": "ordinal",
        "axis": {"orient":"top"},
      "sort": {"op":"sum", "field":"value", "order": "descending"}
    },
    "y": {
      "title": f"Province",
      "field":"province_origine",
      "type": "ordinal",
      "sort": {"op":"sum", "field":"value", "order": "descending"}
    },
    "color": {
      "field": "share",
      "title": "Part (valeur) par province",
      "type": "quantitative",
    }
  },
  "config": {
    "view": {
      "stroke": "transparent"
    }
  }
}, pds.DataFrame(DFLR_product_region_partner_share))

In [ ]:
# extract top by value
nb_top_value = 50
top_provinces = [k for k,v in sorted(total_value_by_province.items(), key=lambda kv:kv[1], reverse=True)[:nb_top_value]]

DFLR_product_region_partner_share=[]
for pd in DFLR_product_region_partner:
  if pd['province_origine'] in top_provinces:
    sort_meta = {
      "share": 100*pd['value']/total_value_by_region[pd['region']],
      "region_sort": top_provinces.index(pd['province_region']) if pd['province_region'] in top_provinces else nb_top_value,
      "province_sort": top_provinces.index(pd['province_origine'])
    }
    DFLR_product_region_partner_share.append(
      dict(list(pd.items())+list(sort_meta.items())))
#DFLR_product_region_partner_share.append({"share":0, "region_sort":0, "province_sort":0, "region": "Lyon", "province_origine": "Lyonnais"})
VegaLite({
  "title": f"Part en valeurs des exports par province d'origine et par direction des fermes en 1789",
  "mark": {"type":"rect", "tooltip": {"content": "data"}},
  "encoding": {
    "x": {
      "field": "region",
      "title": "Directions des fermes",
      "type": "ordinal",
      "sort": {"op":"sum", "field":"value", "order": "descending"},
        "axis": {"orient": "top"}
    },
    "y": {
      "title": "Province d'origine",
      "field":"province_origine",
      "type": "ordinal",
      "sort": {"op":"sum", "field":"value", "order": "descending"}
    },
    "color": {
      "field": "share",
      "title": "share on region",
      "type": "quantitative",
    }
  },
  "config": {
    "view": {
      "stroke": "transparent"
    }
  }
}, pds.DataFrame(DFLR_product_region_partner_share))

# matrice origine partenaire Par Direction


In [ ]:

from itertools import groupby
from collections import defaultdict

product_region_partner = []
total_value_by_region = defaultdict(lambda:0)
by_region = lambda f: f['customs_region']
for region, flows_by_region in groupby(sorted((f for f in all_flows), key = by_region), key = by_region):
    by_province_origin = lambda f: f['origin_province']
    total_value_by_partner = defaultdict(lambda:0)
    total_value_by_province = defaultdict(lambda:0)
    for province, flows_by_province_origin in groupby(sorted(flows_by_region, key=by_province_origin), key=by_province_origin):
        
        by_partner = lambda f: f['partner_grouping']
        for partner, flows_by_province_origin_by_partner in groupby(sorted(flows_by_province_origin, key=by_partner), key=by_partner):
            flows_by_province_origin_by_partner = list(flows_by_province_origin_by_partner)
            
            total_value = sum([float(f['value']) for f in flows_by_province_origin_by_partner if f['value'] != ""])
            by_product = lambda f:f['product_revolutionempire']
            top_product = [f[0] for f in sorted(
                [
                    (product, sum(float(f['value']) for f in flows if f['value'] != "")) 
                        for product,flows in groupby(sorted(flows_by_province_origin_by_partner, key=by_product),key=by_product)
                ], key=lambda f:f[1])][:5]
            total_value_by_province[province] += total_value
            total_value_by_region[region] += total_value
            total_value_by_partner[partner] += total_value
            product_region_partner.append({
                "province_origine": province,
                "region":region,
                
                "partner": partner,
                "value":total_value,
                "top_product": "; ".join(top_product)})
            
    for province, value in total_value_by_province.items():        
        product_region_partner.append({
                "province_origine": province,
                "region":region,
                "partner": "total",
                "value":value
        })
    for partner, value in total_value_by_partner.items():        
        product_region_partner.append({
                "province_origine": "total",
                "region":region,
               
                "partner": partner,
                "value":value
        })
    product_region_partner.append({
                "province_origine": "total",
                "region":region,
               
                "partner": "total",
                "value": total_value_by_region[region]
        })
    
        
product_region_partner.sort(key = lambda f:f['value'],reverse=True)
print("\n".join((f"{f['region']}->{f['province_origine']}->{f['partner']}:{f['value']}" for f in product_region_partner[:20])))

In [ ]:
# extract top by value
display_region = ["La Rochelle","Saint-Malo", "Rouen", "Bordeaux", "Nantes", "Bayonne", "Le Havre", "Marseille"]
region_province_partner_share=[]
for pd in product_region_partner:
    sort_meta = {
      "share": 100*pd['value']/total_value_by_region[pd['region']]
    }
    region_province_partner_share.append(
      dict(list(pd.items())+list(sort_meta.items())))

def origin_partner_matrice(region):
    return VegaLite({
        "title": f"Part des exports 1789 de {region} par province d'origine et destination",
      "mark": {"type":"rect", "tooltip": {"content": "data"}},

      "encoding": {
        "x": {
          "field": "partner",
          "title": "Destination",
          "type": "ordinal",
          "axis": {
              "orient":"top" ,
              "labelAngle":90
            },
          "sort": {"op":"sum", "field":"value", "order": "descending"}
        },
        "y": {
          "title": "province d'origine",
          "field":"province_origine",
          "type": "ordinal",
          "sort": {"op":"sum", "field":"value", "order": "descending"}
        },
        "color": {
          "field": "share",
          "title": "share on region",
          "type": "quantitative",
        }
      },
      "resolve": {"axis": {"x": "independent", "y": "independent"}, "legend": {"color": "independent"}},
      "config": {
        "view": {
          "stroke": "transparent"
        },

      }
    }, pds.DataFrame(f for f in region_province_partner_share if f["region"] == region))

In [ ]:
origin_partner_matrice("La Rochelle")

In [ ]:
origin_partner_matrice("Bordeaux")

In [ ]:
origin_partner_matrice("Nantes")

In [ ]:
origin_partner_matrice("Rouen")

In [ ]:
origin_partner_matrice("Bayonne")

In [ ]:
origin_partner_matrice("Le Havre")